Имеются две базы данных (два листа Excel-файла): база с ценами конкурентов (Data_Parsing) и внутренняя база компании (Data_Company).

В базе парсинга есть два id, однозначно определяющие товар: producer_id и producer_color.

В базе компании есть два аналогичных поля: item_id и color_id.

Нам известно, что коды в двух базах отличаются наличием набора служебных символов. В базе парсинга встречаются следующие символы: _, -, ~, \\, /.

Необходимо:

Считать данные из Excel в DataFrame (Data_Parsing) и (Data_Company).
Подтянуть к базе парсинга данные из базы компании (item_id, color_id, current_price) и сформировать столбец разницы цен в % (цена конкурента к нашей цене).
Определить сильные отклонения от среднего в разности цен в пределах бренда-категории (то есть убрать случайные выбросы, сильно искажающие сравнение). Критерий — по вкусу, написать комментарий в коде.
Записать новый файл Excel с базой парсинга, приклееными к ней столбцами из пункта 2 и с учётом пункта 3 (можно добавить столбец outlier и проставить Yes для выбросов).

In [155]:
import pandas as pd
import numpy as np
import plotly.express as px

In [156]:
data_parsing = pd.read_excel('data/Data_TSUM.xlsx', 'Data_Parsing')
data_company = pd.read_excel('data/Data_TSUM.xlsx', 'Data_Company')

In [157]:
display(data_parsing.head(2), data_company.head(2))

,brand,Category,producer_id,producer_color,price
0,Valentino,Shoes,aaaaa1111_11,black,167
1,Valentino,Shoes,aaaaa1111_12,black,188


,brand,Category,item_id,color_id,current price
0,Valentino,Shoes,aaaaa111111,black,247
1,Valentino,Shoes,aaaaa111112,black,161


In [158]:
def symbol_remover(some_obj):
    symbol_list = ['_', '-', '~', '\\', '/']
    if type(some_obj)==str:
        for s in symbol_list:
            some_obj = some_obj.replace(s, '')
    return some_obj
    

In [159]:
for column in data_parsing.columns:
    data_parsing[column]=data_parsing[column].apply(symbol_remover)


In [160]:
data_parsing['item_id'] = data_parsing['producer_id']

In [161]:
data_parsing.drop('producer_id', axis=1)

,brand,Category,producer_color,price,item_id
0,Valentino,Shoes,black,167,aaaaa111111
1,Valentino,Shoes,black,188,aaaaa111112
2,Valentino,Shoes,black,184,aaaaa111113
3,Valentino,Shoes,black,196,aaaaa111114
4,Valentino,Shoes,black,250,aaaaa111115
...,...,...,...,...,...
70,Stone Island,Bags,red,164,sssss111131
71,Stone Island,Bags,red,158,sssss111132
72,Stone Island,Bags,red,194,sssss111133
73,Stone Island,Bags,red,256,sssss111134


In [162]:
total_data = data_company.merge(data_parsing, how='inner', right_on=['item_id','brand','Category','producer_color'], left_on=['item_id','brand','Category','color_id'])

In [163]:
total_data

,brand,Category,item_id,color_id,current price,producer_id,producer_color,price
0,Valentino,Shoes,aaaaa111111,black,247,aaaaa111111,black,167
1,Valentino,Shoes,aaaaa111112,black,161,aaaaa111112,black,188
2,Valentino,Shoes,aaaaa111113,black,234,aaaaa111113,black,184
3,Valentino,Shoes,aaaaa111114,black,167,aaaaa111114,black,196
4,Valentino,Shoes,aaaaa111115,black,153,aaaaa111115,black,250
...,...,...,...,...,...,...,...,...
70,Stone Island,Bags,sssss111131,red,165,sssss111131,red,164
71,Stone Island,Bags,sssss111132,red,196,sssss111132,red,158
72,Stone Island,Bags,sssss111133,red,236,sssss111133,red,194
73,Stone Island,Bags,sssss111134,red,222,sssss111134,red,256


In [164]:
total_data['difference']=(total_data['current price']-total_data['price'])/total_data['current price']*100
# total_data['difference']=total_data['price']/total_data['current price']*100

In [165]:
total_data['difference'].median()


96.29629629629629

In [166]:
fig_5 = px.box(
            total_data, 
            x="difference",
            # color ='brand',
            title='Распределение клиентов банка по возрасту',
            # labels={'Age':'Возраст клиента', 'Exited': 'Статус'},
            )


fig_5.show()

In [167]:
def outliers_iqr_mod(data, feature, left=1.5, right=1.5, log_scale=True):
    if log_scale:
        x = np.log(data[feature])
    else:
        x = data[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75),
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * left)
    upper_bound = quartile_3 + (iqr * right)
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x >= lower_bound) & (x <= upper_bound)]
    return outliers, cleaned

In [173]:
outliers, cleaned = outliers_iqr_mod(total_data, 'difference', log_scale=False, left=1, right=1)
print(f'Число выбросов по методу z-отклонения: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')

Число выбросов по методу z-отклонения: 6
Результирующее число записей: 69


In [170]:
def outliers_z_score_mod(data, feature, log_scale=False, left=3, right=3):
    if log_scale:
        x = np.log(data[feature]+1)
    else:
        x = data[feature]
    mu = x.mean()
    sigma = x.std()
    lower_bound = mu - left * sigma
    upper_bound = mu + right * sigma
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x >= lower_bound) & (x <= upper_bound)]
    return outliers, cleaned

In [171]:
outliers, cleaned = outliers_z_score_mod(total_data, 'difference', log_scale=False)
print(f'Число выбросов по методу z-отклонения: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')

Число выбросов по методу z-отклонения: 0
Результирующее число записей: 75


In [179]:
fig_6 = px.scatter(total_data, 
                y="price", 
                x="current price", 
                color="Category",
                title='Зависимость стоимости недвижимости от расстояния до Кремля',
                labels={'price_doc':'Стоимсоть недвижимости', 'kremlin_km':'Расстояние до кремля, км'},
                # log_x=True,
                # log_y=True,
                )
fig_6.show()